# Charger les données d'OpenFoodFacts

La base OpenFoodFacts est disponible sous différents formats, dont un export au format CSV que vous avez téléchargé et sur lequel nous allons travailler.

## Documentation
Avant de commencer à travailler sur les données, il est impératif de parcourir sa documentation.
Cette dernière fournit les informations sur le format, indispensables au chargement des données, et sur le contenu, indispensables à la compréhension des données.

La base de données OpenFoodFacts est essentiellement décrite sur [cette page](https://fr-en.openfoodfacts.org/data), un fichier texte fournit la [description des champs](https://static.openfoodfacts.org/data/data-fields.txt).

### Format des données
Si nous appelions naïvement `pandas.read_csv()` avec ses paramètres par défaut, pour charger un fichier aussi gros que le CSV d'OpenFoodFacts (3.1 Go au 2020-12-15), il y a de grandes chances que les choses tournent mal et que pandas remplisse la mémoire de votre machine avant de crasher.
C'est la conséquence du fait que [le format de fichier CSV n'est pas complètement standardisé](https://en.wikipedia.org/wiki/Comma-separated_values).
Le format CSV est plutôt un terme parapluie pour une famille de formats de données à plat, sous forme tabulaire.

La documentation de [read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) et sa liste de paramètres donnent une vision large des principales variantes du format, telles que le choix du caractère utilisé pour séparateur de champs (ou délimiteur), si les noms de colonnes sont écrits sur une ou plusieurs lignes au début du fichier, comment gérer et interpréter les lignes vides, comment gérer et interpréter les éléments entre guillemets, etc.
`read_csv` essaie d'être malin et d'inférer certaines de ces propriétés mais il n'y parvient pas toujours, donc vous devrez le guider et lui fournir les bons paramètres pour votre fichier.

Vous pouvez généralement trouver des informations sur le format et le contenu de votre jeu de données :
* en lisant sa documentation ;
* en exécutant des utilitaires comme [csvkit](https://csvkit.readthedocs.io/en/latest/) ;
* en visualisant le contenu avec un simple programme de visualisation page par page comme [less](https://fr.wikipedia.org/wiki/Less_(Unix)), qui reste très efficace et utile sur les gros fichiers.

Pour le fichier CSV d'OpenFoodFacts :
* Quel est l'encodage du fichier ?

* Quel est le caractère utilisé comme séparateur de champs ?

* Y a-t-il une ou plusieurs lignes d'en-tête (avec les intitulés des colonnes) ?

Quels paramètres devez-vous spécifier à `read_csv` pour charger les données d'OpenFoodFacts ?

## Chargement du jeu de données par tranches
Nous allons commencer par ne charger que les 1000 premières lignes du fichier, afin de jeter un premier coup d'œil au jeu de données et de vérifier son occupation mémoire.
Ensuite, nous lirons les 5000 premières lignes, puis les 25000 premières lignes, en suivant l'occupation mémoire en fonction du nombre de lignes.
Nous ferons ensuite un calcul à la louche afin d'estimer les besoins en mémoire pour charger le jeu de données complet.
C'est sur la base de cette estimation que nous déterminerons les optimisations nécessaires au chargement de tout le jeu de données ou de tout le sous-ensemble qui nous intéresse.

In [11]:
import pandas as pd
# pour les graphes
import seaborn as sns
sns.set()

In [28]:
# TODO modifier le chemin vers le fichier CSV si nécessaire
CSV_FILE = '../data/en.openfoodfacts.org.products.csv'

In [24]:
# lire les 1000 premières lignes

# afficher les 5 premières lignes pour vérifier que la lecture est ok


Examinons maintenant l'occupation mémoire avec [info()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.info.html).

Par défaut, `info()` renvoie un calcul *superficiel* de l'occupation mémoire.
Vous devrez spécifier `memory_usage='deep'` pour déclencher une introspection en mémoire plus profonde qui renvoie une estimation plus juste de l'occupation mémoire.

In [25]:
# TODO

Combien de lignes et de colonnes le DataFrame contient-il?

In [26]:
# TODO

Chargez maintenant les 5000 premières lignes du fichier et notez l'occupation mémoire.

In [ ]:
# lire les données (5000 lignes) et calculer l'occupation mémoire


Chargez maintenant les 25000 premières lignes du fichier et notez l'occupation mémoire.

Pouvez-vous observer une tendance dans l'occupation mémoire? De combien de mémoire auriez-vous besoin pour charger 1.5 million d'entrées ?

## Comment diminuer l'occupation mémoire?

### 1. Ne charger que les colonnes d'intérêt pour l'analyse
`read_csv` accepte un paramètre qui permet de spécifier le sous-ensemble de colonnes que l'on veut charger. Les valeurs des autres colonnes sont jetées immédiatement après avoir été lues, ce qui réduit directement l'occupation mémoire.
Cela semble être une bonne première stratégie pour réduire l'occupation mémoire.

Il est souvent plus facile de déterminer les colonnes que nous ne *voulons pas conserver*.
Ces colonnes appartiennent à deux catégories :
* les colonnes qui ne sont pas directement utiles pour notre but premier, qui est d'analyser les valeurs nutritionnelles, telles que les colonnes contenant des métadonnées de l'entrée dans la base de données (creator, last_modified_t, etc), les lieux de fabrication du produit, le lieu d'achat etc ;
* les colonnes qui sont complètement, ou presque complètement, vides, et qui ne pourront pas contribuer beaucoup à l'analyse.

Il est probable que beaucoup de colonnes d'OpenFoodFacts appartiennent à la deuxième catégorie parce que le schéma de la base de données contient de nombreuses colonnes portant sur des données dont l'affichage n'est pas obligatoire sur les emballages, ou des données qui sont plus difficiles à collecter.

Nous allons vérifier la proportion de cases vides dans chaque colonne avec la fonction [`info`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.info.html).
Comme il y a beaucoup de colonnes, `info` n'affiche par défaut qu'un sommaire minimaliste.
Nous devons spécifier les bons paramètres pour obtenir l'affichage de la liste complète contenant le nom, le type et le nombre de valeurs non-nulles de chaque colonne.

In [27]:
# afficher le type de données et le nombre de valeurs non-nulles dans chaque colonne


Comme vous pouvez le constater :
* Certains champs sont toujours renseignés,
* Certains champs sont parfois renseignés,
* Certains champs ne sont (presque) jamais renseignés.

Combien de champs y a-t-il (environ) dans chacune de ces trois catégories ?
La solution la plus simple est de visualiser la distribution du nombre de valeurs non-nulles dans chaque colonne.

In [ ]:
# visualiser la distribution du nombre de valeurs non-nulles dans chaque colonne


Après exclusion des colonnes de métadonnées et des colonnes (presque) entièrement vides, il nous reste un ensemble de colonnes utilisables pour notre analyse.
Nous pouvons filtrer le jeu de données actuel pour ne conserver que ces colonnes.

In [ ]:
# énumérer les colonnes utilisables
keep_cols = [
    # ...
]
# filtrer les colonnes et afficher le dataframe résultat
df = df[keep_cols]
df

Nous pouvons recharger le fichier CSV en filtrant d'emblée les colonnes utilisables dès `read_csv`.

In [ ]:
# recharger les 25000 premières lignes du fichier CSV en ne gardant que les colonnes utilisables


Ne charger qu'un sous-ensemble des colonnes devrait permettre d'économiser beaucoup de mémoire.
Obtenez l'occupation mémoire de ce Dataframe.

In [ ]:
# calculer l'occupation mémoire


Comparez l'occupation mémoire de ce Dataframe avec celle que vous aviez mesurée précédemment pour le même nombre de lignes, avec toutes les colonnes.

En utilisant le même ratio que précédemment, de quelle quantité de mémoire auriez-vous besoin pour charger toutes les entrées du fichier CSV sur le seul sous-ensemble des colonnes utilisables ?

### 2. Modéliser explicitement les variables catégorielles

Certaines colonnes dans OpenFoodFacts correspondent à des variables catégorielles, par exemple le [NOVA group](https://world.openfoodfacts.org/nova) et la [note Nutri-Score](https://en.wikipedia.org/wiki/Nutri-score).
Les variables catégorielles sont caractérisées par un faible nombre de valeurs possibles, bien inférieur au nombre total d'occurrences de valeurs dans la colonne correspondante d'un grand jeu de données.

pandas a un type de données spécifique, nommé [categorical](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html), avec deux avantages ici :
* l'efficacité mémoire : toutes les instances d'une valeur donnée pointent vers une représentation partagée unique, ce qui économise la mémoire ;
* l'interopérabilité : les variables catégorielles ont une modélisation spécifique dans les bibliothèques d'analyse statistique, d'apprentissage automatique (machine learning) ou de visualisation (graphes différents).

In [20]:
from pandas.api.types import CategoricalDtype

Nous cherchons des colonnes qui pourraient correspondre à des variables catégorielles, en regardant leur nom et le nombre de valeurs uniques dans chaque colonne du jeu de données.

Affichez le nombre de valeurs uniques dans chaque colonne du DataFrame avec [nunique()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.nunique.html).

In [ ]:
# TODO


Distinguez-vous des colonnes avec un très petit nombre de valeurs distinctes, et un nom qui pourrait correspondre à des variables catégorielles ?

Pour chacune de ces colonnes candidates, vérifiez si elle correspond à une variable catégorielle en affichant la liste de ses valeurs uniques avec [unique()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.unique.html).

In [21]:
# TODO

Nous pouvons maintenant déclarer nos variables catégorielles.
La fonction `read_csv` a un paramètre permettant de déclarer le type de données de chaque colonne.

Attention, pandas nomme *catégorielles* des variables à valeurs :
* non ordonnées (variable catégorielle proprement dite),
* ordonnées (variable qualitative ordinale).
Les variables ordinales sont cependant un peu plus compliquées à déclarer proprement.

In [ ]:
# TODO modifier ce dictionnaire pour déclarer proprement les colonnes qui sont
# des variables catégorielles telles que définies ci-dessus
dtype = {
    # string values
    'myColumn025': str,
    # unordered categorical, values will be inferred during reading
    'myColumn123-unordered': 'category',
    # ordered categorical, with the ordered list of values
    'myColumn147-ordered': CategoricalDtype(categories=['first_val', 'second_val', 'third_val'], ordered=True),
}

Rechargez les 25000 premières lignes du fichier CSV, en ne chargeant que les colonnes utilisables, et avec des types de données explicites pour les colonnes correspondant à des variables catégorielles.

In [22]:
# TODO

Vérifiez l'occupation mémoire de ce DataFrame (25000 lignes, colonnes utilisables typées) et comparez-la aux mesures précédentes (25000 lignes toutes colonnes, 25000 lignes colonnes utilisables).

In [ ]:
# TODO

### 3. Diminuer la précision des nombres flottants

Une dernière stratégorie pour réduire l'occupation mémoire est de spécifier des types de données suffisants pour le degré effectif de précision des valeurs.
Par défaut, pandas stocke les nombres réels comme des `float64`, c'est-à-dire en [Double-precision floating-point format](https://en.wikipedia.org/wiki/Double-precision_floating-point_format).

Sur le jeu de données d'OpenFoodFacts, cette approche conservatrice pourrait être considérée comme du gaspillage car les valeurs nutritionnelles écrites sur les emballages ne sont pas d'une précision extrême.
La perte de précision ne devrait pas affecter de façon dramatique l'analyse, donc nous devrions pouvoir utiliser des [`float16`](https://en.wikipedia.org/wiki/Half-precision_floating-point_format) plutôt que des `float64` et diviser par 4 l'occupation mémoire de ces colonnes.

Changez le type des colonnes dont les valeurs correspondent à des quantités de nutriments, en `float16` avec [astype](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html).

In [23]:
# TODO

Examinez les valeurs dans les colonnes numériques dont vous avez changé le type.
Remarquez-vous des changements par rapport aux valeurs stockées précédemment ?

Vérifiez l'occupation mémoire.

In [ ]:
# TODO

Les gains obtenus en diminuant la précision des nombres réels n'étaient pas nécessairement indispensables à ce stade et peuvent vous paraître superflus, voire légèrement perturbants pour les visualisations et les analyses préliminaires.

Diminuer la précision de la représentation des données numériques est surtout utile pour les analyses utilisant des méthodes intensives en calcul, similaires à celles que vous utiliserez dans le notebook 5.

## Conclusion

Nous devrions maintenant être capables de lire tout le jeu de données en spécifiant un sous-ensemble de colonnes qui nous intéresse réellement, et en spécifiant le type des données contenues dans ces colonnes.

* Les variables catégorielles permettent (notamment) d'éviter de stocker de multiples copies de la même valeur.
* Les différents types de données numériques existants permettent d'adapter l'occupation mémoire au degré de précision constaté dans les données ou requis dans les analyses.

## Réutiliser ce travail d'optimisation mémoire

Il ne nous reste plus qu'à écrire ces spécifications de types dans un fichier, ce qui nous permettra de charger efficacement les données dans d'autres notebooks.

In [12]:
DTYPE_FILE = '../data/dtype.txt'
with open(DTYPE_FILE, 'w') as f:
    print(dtype, file=f)

Nous verrons dans le notebook suivant comment filtrer un jeu de données pour travailler sur un sous-ensemble.